In [1]:
import nbformat
from nbformat.v4 import new_code_cell, new_markdown_cell, new_output
from nbclient import NotebookClient
from nbformat.v4 import new_notebook, new_code_cell

# Load the notebook
with open('reproduce-cases-testing.ipynb') as f:
    nb = nbformat.read(f, as_version=4)

new_indexes = []
for i, cell in enumerate(nb.cells):
    if cell.get('outputs'):
        cell = nb.cells[i]
        source = cell.source.splitlines()
        source = [line+"\n" for line in source if line.strip() != '']
        source = [s.replace('\\n', '\n') for s in source]
        s1 = []
        for line in source:
            if line.lstrip()[0] != "#":
                s1.append(line)
        if s1[-1][0] != " ":
            new_cell = new_code_cell()
            new_cell.source = "type(("+s1[-1].rstrip()+"))\n"
            nb.cells.insert(i+1, new_cell)
            new_indexes.append(i+1)
print()
# Save the modified notebook
with open('new_notebook.ipynb', 'w') as f:
    nbformat.write(nb, f)


In [2]:
!jupyter nbconvert --execute --to notebook --inplace new_notebook.ipynb

[NbConvertApp] Converting notebook new_notebook.ipynb to notebook
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
[NbConvertApp] Writing 17872 bytes to new_notebook.ipynb


In [3]:
import json
with open("new_notebook.ipynb", "r") as f:
    nb1 = nbformat.read(f, as_version=4)

In [4]:
import json 
with open("reproduce-cases.ipynb", mode = "r", encoding= "utf-8") as f:
    fist = nbformat.read(f, as_version=4)

In [5]:
new_indexes

[2, 5, 7, 9, 11, 13, 15, 18, 21, 23]

In [6]:
types = []
testing_outputs = []
for i, ce in enumerate(nb1.cells):
    if i in new_indexes and ce["cell_type"] == "code":
        types.append(ce["outputs"][0]["data"]["text/plain"])
    elif ce["cell_type"] == "code" and ce.get("outputs"):
        testing_outputs.append(ce["outputs"][0]["data"]["text/plain"])

In [7]:
testing_outputs

["('weather-buoy-data.txt.gz', <http.client.HTTPMessage at 0x1054bea90>)",
 'array([[88.        , 88.09090909, 88.18181818, ..., 88.        ,\n        88.09090909, 88.18181818],\n       [88.27272727, 88.36363636, 88.45454545, ..., 88.27272727,\n        88.36363636, 88.45454545],\n       [88.54545455, 88.63636364, 88.72727273, ..., 88.54545455,\n        88.63636364, 88.72727273],\n       ...,\n       [88.81818182, 88.        , 88.09090909, ..., 88.81818182,\n        88.        , 88.09090909],\n       [88.18181818, 88.27272727, 88.36363636, ..., 88.18181818,\n        88.27272727, 88.36363636],\n       [88.45454545, 88.54545455, 88.63636364, ..., 88.45454545,\n        88.54545455, 88.63636364]])',
 'array([[88.  , 88.09, 88.18, ..., 88.  , 88.09, 88.18],\n       [88.27, 88.36, 88.45, ..., 88.27, 88.36, 88.45],\n       [88.55, 88.64, 88.73, ..., 88.55, 88.64, 88.73],\n       ...,\n       [88.82, 88.  , 88.09, ..., 88.82, 88.  , 88.09],\n       [88.18, 88.27, 88.36, ..., 88.18, 88.27, 88.36

In [7]:
original_outputs = []

for i, ce in enumerate(fist.cells):
    if ce.get('outputs'):
        original_outputs.append((ce["outputs"][0]["data"]["text/plain"], i))

In [8]:
len(types) == len(original_outputs) == len(testing_outputs)

True

In [9]:
types

['tuple',
 'numpy.ndarray',
 'numpy.ndarray',
 'numpy.ndarray',
 'numpy.ndarray',
 'bool',
 'tuple',
 'tuple',
 'int',
 'list']

In [10]:
new_list = list()

def compare_numpy_arrays(arr1, arr2):
    d= {}
    try:
        if arr1[0][0:5] == "array" and arr2[0][0:5] == "array":
            for u in range(len(arr1)):
                arr1[u] = arr1[u].replace("...,", "").strip()
            un = "".join(arr1)
#             print(un)
            lst_arr = np.array(eval("np." + un))
            new_arr = np.delete(np.delete(lst_arr,3,axis =0), -4, axis =0)
            new_arr = np.delete(np.delete(new_arr,3, axis = 1), -4, axis=1)
            for u in range(len(arr2)):
                arr2[u] = arr2[u].replace("...,", "").strip()
            un = "".join(arr2)
            lst_arr1 = np.array(eval("np." + un))
            mae = mean_absolute_error(new_arr, lst_arr1)
            return f"{mae} mae value"

        else:
            return "Not an numpy array"
    except Exception as e:
        print(e)
        return e

In [11]:
def compare_tuples(t1, t2):
    score = 0
    for i in range(len(t1)):
        print(type(t1[i]), t1[i])
        if isinstance(t1[i], str) and isinstance(t2[i], str):
            if t1[i] == t2[i]:
                score += 1
        elif isinstance(t1[i], (list, tuple)) and isinstance(t2[i], (list, tuple)):
            score += compare_tuples(t1[i], t2[i])
        elif isinstance(t1[i], (int, float)) and isinstance(t2[i], (int, float)):
            if abs(t1[i] - t2[i]) < 0.0001:
                score += 1
    return score / len(t1)

In [12]:
def compare_strings(s, s1,i,j):
    # s = first_dict[13][1][:i+1]
    # s1 = second_dict[13][1][:j+1]
    # s1 = [i.replace("\\n", "\n").replace("[", "").replace("]", "").replace("'", "").replace(",", "").strip() for i in s1]
    # s = [i.replace("\n", "").replace("[", "").replace("]", "").replace("'", "").replace(",", "").strip() for i in s]
    li = []
    for line in s:
        d = dict()
        le = 0
        # li.append(Counter(line.split()))
        for count, word in enumerate(line.split()):
            le = le + len(word)
            if word in d.keys():
                # d[word][0] += 1
                # d[word][1].add(count+1)
                d[word][1].append((le-len(word), le))
            else:
                # d[word] = [1, {count+1}]
                d[word] = [1,[(le-len(word),le)]]
            
            le = le+1
        li.append(d)
    # print(li)
    cw = set(li[0].keys())
    for i in li[1:]:
        cw = cw.intersection(set(i.keys()))
    mcw = dict()
    for i in li:
        for k in i:
            if k in mcw.keys():
                mcw[k][0] += i[k][0]
                mcw[k][1].append(i[k][1])
            else:
                mcw[k] = [i[k][0], [i[k][1]]]
    # print(mcw["Tropical"])
    for k in mcw:
        ol = mcw[k]
        nl = [ol[0]]
        tl = []
        for sublist in ol[1]:
            tl += sublist
        nl.append(tl)
        nl[1] = list(set(nl[1]))
        mcw[k] = nl
    import re
    cwd = dict()
    for word in cw:
        cwd[word] = mcw[word]
    whole_percentage = []
    for count, st in enumerate(s1):
        # print(st)
        percentage = 0
        for word in cwd:
            strr = r"" + word
            reg = r"\b" + strr + r"\b"
            indexes = re.search(reg, st)
            # print(word, indexes)
            if cwd[word] is not None and indexes is not None:
                if  (indexes.start(), indexes.end()) in cwd[word][1]:
                    percentage +=1
        whole_percentage.append(percentage/len(cwd))
    return sum(whole_percentage) / len(whole_percentage)
    

In [13]:
import ast
import re
import numpy as np
import numpy as np
from sklearn.metrics import mean_absolute_error
res_dict = dict()

for i, (ty, out1, out2) in enumerate(zip(types, original_outputs, testing_outputs)):
    if ty == "tuple":
        y = out1[0]
        x = out2
        pattern = r'<.*?>'
        match = re.search(pattern, x)
        match1 = re.search(pattern, y)
        if match and match1:
            obj_str = match.group(0)
            obj_str1 = match1.group(0)
            x = x.replace(obj_str, str("'" + obj_str.split(" ")[0][1:]+ "'"))
            y = y.replace(obj_str1, str("'" + obj_str1.split(" ")[0][1:] + "'"))            
        first_output = ast.literal_eval(x)
        second_output = ast.literal_eval(y)
        ret = compare_tuples(first_output,second_output)
        res_dict[(out1[1], out1[1])] = ret
    elif ty == "int":
        ii = out1[0]
        jj = out2
        if int(ii) == int(jj):
            res_dict[(out1[1], out1[1])] = 1
        else:
            res_dict[(out1[1],out1[1])] = 0 
    elif ty == "list":
        s = out1[0]
        s1 = out2
        oi =  ast.literal_eval(s)
        oi2 = ast.literal_eval(s1)
        list_types_s = []
        list_types_s1 = []
        list1_str_index = 0
        list2_str_index = 0
        for t in oi:
            if isinstance(t, str):
                list1_str_index += 1
            else:
                break
        for t in oi2:
            if isinstance(t, str):
                list2_str_index +=1
            else:
                break
        res2 = compare_strings(oi[:list1_str_index], oi2[:list2_str_index], list1_str_index, list2_str_index)
        res_dict[(out1[1], out1[1])] = res2
    elif ty  == "numpy.ndarray":
        arr1 = out1[0].strip().split("\n")
        arr2 = out2.strip().split("\n")
        res_dict[(out1[1], out1[1])] = compare_numpy_arrays(arr1, arr2)
    elif ty is bool:
        if first_dict[du][1] == second_dict[du1][1]:
             res_dict[(out1[1], out1[1])] = 0
        else:
            res_dict[(out1[1], out1[1])] = 1
        

<class 'str'> weather-buoy-data.txt.gz
<class 'str'> http.client.HTTPMessage
<class 'tuple'> (16, 2333)
<class 'int'> 16
<class 'int'> 2333
<class 'tuple'> (16, 2333)
<class 'int'> 16
<class 'int'> 2333
<class 'tuple'> (16, 2332)
<class 'int'> 16
<class 'int'> 2332
<class 'str'> fl-weather-alerts.json
<class 'str'> http.client.HTTPMessage


In [14]:
res_dict

{(1, 1): 1.0,
 (3, 3): '2.6794444446901857e-06 mae value',
 (4, 4): '0.0 mae value',
 (5, 5): '0.0 mae value',
 (6, 6): '0.0 mae value',
 (8, 8): 1.0,
 (10, 10): 1.0,
 (12, 12): 0,
 (13, 13): 0.5809523809523811}